In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("../data/prediksi/prediksi_liu.csv", sep=";")

In [3]:
data

,Tweet,label,Tweet_Parsed,Prediksi,Prediksi_Score
0,"Min, normal ke kalau iPhone 11/11 Pro /Pro Max...",baterai negatif,admin normal pakai baterai kapasitas turun,baterai positif,baterai 0
1,Kmera iphone 11 mmeng beda,kamera positif,kamera beda,kamera positif,kamera 0
2,Iphone 11 pro max kamera depannya jelek ye? Gu...,kamera negatif,kamera depannya jelek liat liat pengguna foto ...,kamera negatif,kamera -1
3,Apple meluncurkan iPhone 11 dengan peningkatan...,baterai positif,meluncurkan peningkatan daya tahan baterai leb...,baterai positif,baterai 1
4,"xs max, senderrr. tapi rugi sih kalo kamu beli...","spesifikasi positif,harga positif",rugi beli maksud langsung tidak speknya lebih ...,"spesifikasi negatif,harga positif","spesifikasi -2,harga 1"
...,...,...,...,...,...
743,ga keren banget masa charger bawaannya belum f...,baterai negatif,tidak keren sekali charger bawaannya belum cep...,baterai positif,baterai 1
744,"yaelah belum fast charging, lama nih kalo ngec...",baterai negatif,yaelah belum cepat charger lama charge charger...,baterai positif,baterai 1
745,butuh waku lama buat ngecharge kalo pake charg...,baterai negatif,butuh waku lama charge charger bawaan daya mas...,baterai negatif,baterai -1
746,udah mahal begini masa chargernya blm fast cha...,baterai negatif,udah mahal chargernya belum cepat charger kecewa,"harga negatif,baterai positif","harga -2,baterai 1"


In [4]:
class eval:
    def __init__(self):
        pass
    
    def aspect_count(self,data, aspect, label_col = 'label'):
        label_list = data[label_col].values
        pos, neg = 0, 0
        for i in label_list:
            split = i.split(",")
            for j in split:
                in_split = j.split(" ")
                if in_split[0] == aspect:
                    if in_split[1] == 'negatif':
                        neg = neg+1
                    else:
                        pos = pos+1
        return pos,neg
    
    def eval_count(self,data,true_label_col='label',pred_label_col='Prediksi',aspect_list=['general','spesifikasi','kamera','baterai','harga']):
        label_list = [str(i).split(",") for i in data[true_label_col].values]
        pred_list = [str(i).split(",") for i in data[pred_label_col].values]
        sent = ['positif','negatif']
        eval_list = []
        for i in aspect_list:
            for j in range(len(sent)):
                asp_count = self.aspect_count(data,i)
                record = [i,sent[j],asp_count[j]]
                pred_count = 0
                pred_true = 0
                pred_false = 0
                for k,l in zip(label_list,pred_list):
                    if " ".join([i,sent[j]]) in l:
                        pred_count = pred_count+1
                        if " ".join([i,sent[j]]) in k:
                            pred_true = pred_true+1
                        else:
                            pred_false = pred_false+1
                record.extend([pred_count,pred_true,pred_false])
                eval_list.append(record)
        return eval_list
    
    def get_pred_text(self,data,true_label_col='label',pred_label_col='Prediksi',text_col="Tweet",prep_text_col='Tweet_Parsed',predicted = False):
        label_list = data[true_label_col].values
        pred_list = data[pred_label_col].values
        text_list = data[text_col].values
        prep_text_list = data[prep_text_col].values
        pred_text = []
        for i in range(data.shape[0]):
            if predicted == False:
                if label_list[i] != pred_list[i]:
                    pred_text.append([text_list[i],prep_text_list[i],label_list[i],pred_list[i]])
            else:
                if label_list[i] == pred_list[i]:
                    pred_text.append([text_list[i],prep_text_list[i],label_list[i],pred_list[i]])
                
        return pred_text

In [5]:
evaluate = eval()

In [6]:
eval_result = evaluate.eval_count(data)
eval_result = pd.DataFrame(eval_result,columns=['aspect','sentiment','label count','predicted count','true predict','false predict'])
eval_result = eval_result.groupby(['aspect','sentiment']).sum()
eval_result['percent_true'] = round((eval_result['true predict']/eval_result['label count'])*100,2)
eval_result['percent_predict_true'] = round((eval_result['true predict']/eval_result['predicted count'])*100,2)
eval_result

label count  predicted count  true predict  \
aspect      sentiment                                               
baterai     negatif             77               79            50   
            positif             82               85            55   
general     negatif             82               94            46   
            positif            107              157            86   
harga       negatif             79               77            51   
            positif             65               64            41   
kamera      negatif             70               57            38   
            positif            110              119            84   
spesifikasi negatif             62               35            20   
            positif             86               81            57   

                       false predict  percent_true  percent_predict_true  
aspect      sentiment                                                     
baterai     negatif               29         64.94                 63.29  
            positif               30         67.07                 64.71  
general     negatif               48         56.10                 48.94  
            positif               71         80.37                 54.78  
harga       negatif               26         64.56                 66.23  
            positif               23         63.08                 64.06  
kamera      negatif               19         54.29                 66.67  
            positif               35         76.36                 70.59  
spesifikasi negatif               15         32.26                 57.14  
            positif               24         66.28                 70.37

In [7]:
false_tweet = evaluate.get_pred_text(data)
false_tweet = pd.DataFrame(false_tweet,columns=['Tweet','Tweet_Parsed','True_Label','Pred_Label'])
false_tweet

,Tweet,Tweet_Parsed,True_Label,Pred_Label
0,"Min, normal ke kalau iPhone 11/11 Pro /Pro Max...",admin normal pakai baterai kapasitas turun,baterai negatif,baterai positif
1,"xs max, senderrr. tapi rugi sih kalo kamu beli...",rugi beli maksud langsung tidak speknya lebih ...,"spesifikasi positif,harga positif","spesifikasi negatif,harga positif"
2,Saya juga dulu confuse nak pilih iphone 11 or ...,bingung ingin pilih review spesifikasi kurang ...,kamera negatif,"spesifikasi negatif,general negatif,kamera pos..."
3,"tahun kemarin pas iphone 11 rilis, mata langsu...",kemarin rilis mata langsung warnanya warna ung...,general positif,general negatif
4,Saya suka iPhone11 Pro. Tapi tak mampu. Oleh i...,suka tidak mampu tunggu harap rezeki,"general positif,harga negatif",general negatif
...,...,...,...,...
310,udah coba pake handphone ini seharian tanpa ng...,udah coba handphone seharian tanpa charge waw ...,baterai positif,baterai negatif
311,ga keren banget masa charger bawaannya belum f...,tidak keren sekali charger bawaannya belum cep...,baterai negatif,baterai positif
312,"yaelah belum fast charging, lama nih kalo ngec...",yaelah belum cepat charger lama charge charger...,baterai negatif,baterai positif
313,udah mahal begini masa chargernya blm fast cha...,udah mahal chargernya belum cepat charger kecewa,baterai negatif,"harga negatif,baterai positif"


In [8]:
true_tweet = evaluate.get_pred_text(data,predicted=True)
true_tweet = pd.DataFrame(true_tweet,columns=['Tweet','Tweet_Parsed','True_Label','Pred_Label'])
true_tweet

,Tweet,Tweet_Parsed,True_Label,Pred_Label
0,Kmera iphone 11 mmeng beda,kamera beda,kamera positif,kamera positif
1,Iphone 11 pro max kamera depannya jelek ye? Gu...,kamera depannya jelek liat liat pengguna foto ...,kamera negatif,kamera negatif
2,Apple meluncurkan iPhone 11 dengan peningkatan...,meluncurkan peningkatan daya tahan baterai leb...,baterai positif,baterai positif
3,Hasil kamera xiaomi note 10 bisa di adulah sam...,hasil kamera xiaomi note bisa adulah lebih baik,kamera positif,kamera positif
4,Kalo sender lebih mentingin ketajaman layar me...,lebih lebih ketajaman layar lebih baik keselur...,"general positif,kamera positif,spesifikasi pos...","general positif,kamera positif,spesifikasi pos..."
...,...,...,...,...
428,harus banget bawa powerbank kemana mana ini ma...,harus sekali bawa powerbank kemana mah baterai...,baterai negatif,baterai negatif
429,kenapa sih iphone tuh baterainya ga pernah ber...,tuh baterainya tidak bertahan lama,baterai negatif,baterai negatif
430,"baterai nya boros, ga cocok buat orang yang pe...",baterai boros tidak cocok orang pekerjaannya h...,baterai negatif,baterai negatif
431,ngecharge nya butuh watu lama ini iphone 11,charge butuh watu lama,baterai negatif,baterai negatif


In [9]:
eval_result.to_csv("../data/final/evalresult_liu.csv", sep=";", index=False)
false_tweet.to_csv("../data/final/falsetweet_liu.csv", sep=";", index=False)
true_tweet.to_csv("../data/final/truetweet_liu.csv", sep=";", index=False)